In [11]:
import tensorflow as tf
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.python.ops import io_ops
import math
import time


In [17]:
image = io_ops.read_file("bio0000_frame0000001.jpg")
color = tf.io.decode_png(image,channels=3,dtype=tf.dtypes.uint8,name=None)

image = io_ops.read_file("bio0000_frame0000001.jpg")
color1 = tf.io.decode_png(image,channels=3,dtype=tf.dtypes.uint8,name=None)

In [18]:
img = tf.image.convert_image_dtype(color, tf.dtypes.float32)
img = tf.image.rgb_to_grayscale(img)
img = tf.image.convert_image_dtype(img, tf.dtypes.uint8)
gray = tf.squeeze(img,2)

img = tf.image.convert_image_dtype(color1, tf.dtypes.float32)
img = tf.image.rgb_to_grayscale(img)
img = tf.image.convert_image_dtype(img, tf.dtypes.uint8)
gray1 = tf.squeeze(img,2)

# Otsu Thresholding

In [21]:
def otsu_thresholding(image):
    image = tf.convert_to_tensor(image, name="image")

    rank = image.shape.rank
    if rank != 2 and rank != 3:
        raise ValueError("Image should be either 2 or 3-dimensional.")

    if image.dtype!=tf.int32:
        image = tf.cast(image, tf.int32)

    r, c = image.shape
    hist = tf.math.bincount(image, dtype=tf.int32)
    
    if len(hist)<256:
        hist = tf.concat([hist, [0]*(256-len(hist))], 0)

    current_max, threshold = 0, 0
    total = r * c

    spre = [0]*256
    sw = [0]*256
    spre[0] = int(hist[0])

    for i in range(1,256):
        spre[i] = spre[i-1] + int(hist[i])
        sw[i] = sw[i-1]  + (i * int(hist[i]))

    for i in range(256):
        if total - spre[i] == 0:
            break

        meanB = 0 if int(spre[i])==0 else sw[i]/spre[i]
        meanF = (sw[255] - sw[i])/(total - spre[i])
        varBetween = (total - spre[i]) * spre[i] * ((meanB-meanF)**2)

        if varBetween > current_max:
            current_max = varBetween
            threshold = i

    final = tf.where(image>threshold,255,0)
    return final

In [22]:
out3 = otsu_thresholding(gray)
plt.imshow(out3, cmap="gray")

TypeError: object of type 'Tensor' has no len()

# Histogram Equalization

In [1]:
import tensorflow as tf
import numpy as np
import cv2

def tf_equalize_histogram(image):
    values_range = tf.constant([0., 255.], dtype = tf.float32)
    histogram = tf.histogram_fixed_width(tf.to_float(image), values_range, 256)
    cdf = tf.cumsum(histogram)
    cdf_min = cdf[tf.reduce_min(tf.where(tf.greater(cdf, 0)))]

    img_shape = tf.shape(image)
    pix_cnt = img_shape[-3] * img_shape[-2]
    px_map = tf.round(tf.to_float(cdf - cdf_min) * 255. / tf.to_float(pix_cnt - 1))
    px_map = tf.cast(px_map, tf.uint8)

    eq_hist = tf.expand_dims(tf.gather_nd(px_map, tf.cast(image, tf.int32)), 2)
    return eq_hist

image_ph = tf.placeholder(tf.uint8, shape = [None, None, 1])
image_eq_hist = tf_equalize_histogram(image_ph)

image = cv2.imread("bio0000_frame0000001.jpg", 0)
image = np.reshape(image, (image.shape[0], image.shape[1], 1))
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    image_eq_hist_ = sess.run(image_eq_hist, feed_dict = {image_ph : image})

cv2.imshow("eq_cv", cv2.equalizeHist(image))
cv2.waitKey()

cv2.imshow("eq", image_eq_hist_)
cv2.waitKey()



C:\Users\DELL\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\DELL\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\DELL\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\DELL\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (

Instructions for updating:
Use `tf.cast` instead.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


50